In [1]:
import pyranges as pr
import pandas as pd
import numpy as np
from define_helpers import select_rep_site, _define_cryptic_status, select_rep_prox_site
import os

In [2]:
# input novel last exons used to generate combined reference of last exons
novel_le = pr.read_gtf("../data/papa/2023-03-29_papa_i3_cortical_upf1_zanovello_overlap_annotated.gtf")
# dexseq results df (used to extract cryptic events of each class)
dexseq_df = pd.read_csv("../data/papa/2023-05-24_i3_cortical_zanovello.all_datasets.dexseq_apa.results.processed.cleaned.tsv", sep="\t")
# refeence GTF used to identify novel last exons, quantify vs ref
ref_gtf = pr.read_gtf("../data/reference_filtered.gtf")
# 
tx2le = pd.read_csv("../data/papa/novel_ref_combined.tx2le.tsv", sep="\t")

# last exon quantification regions used as input to Salmon
quant_uniq_le = pr.read_gtf("../data/papa/novel_ref_combined.quant.last_exons.gtf")


info_df = pd.read_csv("../data/papa/novel_ref_combined.info.tsv", sep="\t")

In [3]:
# get le_ids that are novel bleedthroughs or ALEs
# le_id_spliced = set(dexseq_df[dexseq_df.simple_event_type == "distal_3utr_extension"].le_id)
# le_id_bleed = set(dexseq_df[dexseq_df.simple_event_type == "distal_3utr_extension"].le_id)
dexseq_df.simple_event_type.value_counts()


simple_event_type
spliced                  130806
bleedthrough              41453
distal_3utr_extension     37225
Name: count, dtype: int64

In [4]:
le_id_spliced = dexseq_df.loc[dexseq_df.simple_event_type == "spliced", "le_id"]
le_id_bleed = dexseq_df.loc[dexseq_df.simple_event_type == "bleedthrough", "le_id"]
le_id_d3utr = dexseq_df.loc[dexseq_df.simple_event_type == "distal_3utr_extension", "le_id"]


In [5]:
dexseq_df[dexseq_df["gene_name"] == "SIN3B"]

,experiment_name,binID,groupID,featureID,exonBaseMean,dispersion,stat,pvalue,padj,UsageCoefficient_base,...,annot_status,transcript_id,chromosome,strand,start,end,mean_PPAU_base,mean_PPAU_treatment,delta_PPAU_treatment_control,simple_event_type
40,brown_i3_cortical,ENSG00000127511.10:E001,ENSG00000127511.10,E001,685.073107,0.013534,127.151527,1.721175e-29,3.384311e-27,10.215189,...,novel,PAPA.ctrl_ctrl_2.24194.4,chr19,+,1.685414e+07,1.685742e+07,0.065536,0.226421,0.160885,bleedthrough
41,brown_i3_cortical,ENSG00000127511.10:E002,ENSG00000127511.10,E002,1635.679342,0.013184,130.260073,3.594567e-30,7.236205e-28,17.008705,...,annotated,"ENST00000601141.5,ENST00000595541.1,ENST000005...",chr19,+,1.687850e+47,1.687906e+47,0.832218,0.649472,-0.182746,spliced
16839,brown_shsy5y,ENSG00000127511.10:E001,ENSG00000127511.10,E001,646.630524,0.070550,0.725662,3.942930e-01,8.831562e-01,13.366541,...,novel,PAPA.ctrl_ctrl_2.24194.4,chr19,+,1.685414e+07,1.685742e+07,0.181492,0.147692,-0.033800,bleedthrough
16840,brown_shsy5y,ENSG00000127511.10:E002,ENSG00000127511.10,E002,1583.366570,0.069263,0.738940,3.900008e-01,8.809464e-01,16.186991,...,annotated,"ENST00000601141.5,ENST00000595541.1,ENST000005...",chr19,+,1.687850e+47,1.687906e+47,0.789384,0.806265,0.016880,spliced
25677,brown_skndz,ENSG00000127511.10:E001,ENSG00000127511.10,E001,123.423529,0.990241,0.184085,6.678856e-01,8.576725e-01,7.870387,...,annotated,ENST00000596802.5,chr19,+,1.685414e+07,1.685574e+07,0.091220,0.082138,-0.009082,bleedthrough
25678,brown_skndz,ENSG00000127511.10:E002,ENSG00000127511.10,E002,1382.957785,0.017284,3.819419,5.066181e-02,2.041506e-01,18.595389,...,novel,PAPA.ctrl_ctrl_2.24194.4,chr19,+,1.685414e+07,1.685742e+07,0.459231,0.371105,-0.088125,bleedthrough
25679,brown_skndz,ENSG00000127511.10:E003,ENSG00000127511.10,E003,734.474029,0.013301,5.624554,1.771058e-02,1.009837e-01,14.791017,...,annotated,"ENST00000601141.5,ENST00000595541.1,ENST000005...",chr19,+,1.687850e+47,1.687906e+47,0.436124,0.489640,0.053516,spliced
34381,humphrey_i3_cortical,ENSG00000127511.10:E001,ENSG00000127511.10,E001,31.461331,0.988492,3.663168,5.562777e-02,1.940285e-01,4.173276,...,annotated,ENST00000596802.5,chr19,+,1.685414e+07,1.685574e+07,0.019720,0.072751,0.053031,bleedthrough
34382,humphrey_i3_cortical,ENSG00000127511.10:E002,ENSG00000127511.10,E002,232.592245,0.020956,177.904729,1.389688e-40,7.563611e-38,6.775476,...,novel,PAPA.ctrl_ctrl_2.24194.4,chr19,+,1.685414e+07,1.685742e+07,0.062796,0.271946,0.209150,bleedthrough
34383,humphrey_i3_cortical,ENSG00000127511.10:E003,ENSG00000127511.10,E003,469.902739,0.024516,174.525798,7.599079e-40,3.877430e-37,9.311451,...,annotated,"ENST00000601141.5,ENST00000595541.1,ENST000005...",chr19,+,1.687850e+47,1.687906e+47,0.908924,0.596359,-0.312565,spliced


In [6]:
# good idea to ensure don't consider spliced LEs that have a 3'UTR extension
# UPDATE now need to select representative proximal site for each distal site, since quantification approach lumps together all annotated proximal sites (and compares them to extension) 
# since le_ids are annotated sequentially, know that immediately succeeding le_number is the partner le_id
le_id_spliced_spl = le_id_spliced.str.split("_", regex=False, expand=True)
le_id_spliced_spl[1] = le_id_spliced_spl[1].astype(int).add(1)
# reconstruct le_id (corresponding to theoretical distal 3'UTR extension le_id)
le_id_spliced_d3utr = le_id_spliced_spl[0].str.cat(le_id_spliced_spl[1].astype(str), sep="_")

# now have theoretical distal 3'UTR le_ids, intersect with actual ids - any overlapping will be removed from downstream analysis
spliced_d3utr_olap = le_id_spliced_d3utr.isin(le_id_d3utr.values)
print(f"Number of spliced last exons with 3'UTR extension - {spliced_d3utr_olap.sum()}")

# now use mask to remove these ALEs
le_id_spliced = le_id_spliced[~spliced_d3utr_olap]
le_id_spliced


Number of spliced last exons with 3'UTR extension - 58895


0         ENSG00000021645.20_1
1         ENSG00000021645.20_4
2         ENSG00000021645.20_4
3         ENSG00000048649.14_1
4         ENSG00000048649.14_1
                  ...         
209475     ENSG00000278311.5_5
209480     ENSG00000280832.2_1
209481     ENSG00000280832.2_2
209482     ENSG00000282508.2_4
209483     ENSG00000282508.2_6
Name: le_id, Length: 71911, dtype: object

In [7]:
# add le_id used in downstream analysis
novel_le = novel_le.apply(lambda df: df.merge(tx2le, on="transcript_id", how="inner", suffixes=[None, "_quant"]))


# Select representative PAS for bleedthroughs and spliced events (novel)
novel_le_rep_spliced, rep_choices_spliced = select_rep_site(novel_le.subset(lambda df: df.le_id_quant.isin(set(le_id_spliced))), id_col="le_id_quant")


print(f"Number of spliced intervals before selecting representative LEs - {len(novel_le.subset(lambda df: df.le_id_quant.isin(set(le_id_spliced))))}")
print(f"Number of spliced intervals after selecting representative LEs - {len(novel_le_rep_spliced)}")
print({dec: len(ids) for dec, ids in rep_choices_spliced.items()})




Number of spliced intervals before selecting representative LEs - 6209
Number of spliced intervals after selecting representative LEs - 949
{'atlas_1_pred': 571, 'atlas_max_datasets': 143, 'atlas_max_datasets_shortest': 98, 'motif_1_min': 321, 'motif_shortest_min': 160}


In [8]:
# Select representative PAS for bleedthroughs and spliced events
novel_le_rep_bleed, rep_choices_bleed = select_rep_site(novel_le.subset(lambda df: df.le_id_quant.isin(set(le_id_bleed))), id_col="le_id_quant")

print(f"Number of bleedthrough intervals before selecting representative LEs - {len(novel_le.subset(lambda df: df.le_id_quant.isin(set(le_id_bleed))))}")
print(f"Number of spliced intervals after selecting representative LEs - {len(novel_le_rep_bleed)}")
print({dec: len(ids) for dec, ids in rep_choices_bleed.items()})

Number of bleedthrough intervals before selecting representative LEs - 5952
Number of spliced intervals after selecting representative LEs - 931
{'atlas_1_pred': 590, 'atlas_max_datasets': 152, 'atlas_max_datasets_shortest': 105, 'motif_1_min': 350, 'motif_shortest_min': 114}


In [9]:
# now want to generate an ID with minimal info required for making maps
# 3'end coordinates/last exon, gene name, le_id, site type, regulation status
# site_type - spliced/bleedthrough
# regulation_status - cryptic/background


# define cryptic le_ids
# define cryptic gene_ids
dexseq_df_cryp = dexseq_df[(dexseq_df["padj"].lt(0.05)) & (dexseq_df["mean_PPAU_base"].le(0.1)) & (dexseq_df["delta_PPAU_treatment_control"].ge(0.1))]
cryp_le_ids = set(dexseq_df_cryp.le_id)
cryp_gene_names = set(dexseq_df_cryp.gene_name)


# define cryptic & background le_ids
# background - ns in any dataset (to do this, get list of sig in any dataset)
# get gene IDs with no regulated ALEs in any dataset
# returns pd.Series
ns_gene_ids = (dexseq_df.assign(reg_status=lambda df: np.where(df["padj"].le(0.05), 1, 0))
               .groupby("groupID")
               ["reg_status"]
               .sum()
               .loc[lambda x: x == 0]
             )

# set of le_ids that belong to genes with no significant ALEs (is this too conservative?)
ns_le_ids = set(dexseq_df.loc[dexseq_df["groupID"].isin(ns_gene_ids.index), "le_id"])


print(f"Number of cryptic ALE-containing genes - {len(cryp_gene_names)}")
print(f"Number of cryptic ALE isoforms - {len(cryp_gene_names)}")

print(f"Number of ns ALE-containing genes - {len(ns_gene_ids)}")
print(f"Number of ns ALE isoforms - {len(ns_le_ids)}")


Number of cryptic ALE-containing genes - 283
Number of cryptic ALE isoforms - 283
Number of ns ALE-containing genes - 2937
Number of ns ALE isoforms - 6365


In [10]:
# assign cryptic status for novel bleedthroughs
novel_le_rep_bleed = novel_le_rep_bleed.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids, ns_le_ids, id_col="le_id_quant"))
novel_le_rep_bleed.cryptic_status.value_counts()


cryptic_status
NULL          619
background    285
cryptic        27
Name: count, dtype: int64

In [11]:
# assign cryptic status for novel spliced events
novel_le_rep_spliced = novel_le_rep_spliced.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids, ns_le_ids, id_col="le_id_quant"))
novel_le_rep_spliced.cryptic_status.value_counts()

cryptic_status
NULL          604
background    306
cryptic        39
Name: count, dtype: int64

In [12]:
# Construct 'name' string with minimal annotation information
# le_id|gene_name|site_type|cryptic_status

novel_le_rep_spliced = (novel_le_rep_spliced
 .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("gene_name_ref",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         # some ref_gene_name entries have multiple gene names
         lambda df: df["gene_name_ref"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
         .assign("site_type",
                 lambda df: pd.Series(["spliced"]*len(df.index), index=df.index))
 .assign("Name",
                          lambda df: df["le_id_quant"].str.cat(df[["gene_name_ref", "site_type", "cryptic_status"]], sep="|"))
                    
)

novel_le_rep_bleed = (novel_le_rep_bleed
 .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("gene_name_ref",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         # some ref_gene_name entries have multiple gene names
         lambda df: df["gene_name_ref"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
         .assign("site_type",
                 lambda df: pd.Series(["bleedthrough"]*len(df.index), index=df.index))
 .assign("Name",
                          lambda df: df["le_id_quant"].str.cat(df[["gene_name_ref", "site_type", "cryptic_status"]], sep="|"))
                    
)

In [13]:
# for spliced events, want to report the last exon coordinates (also split into 5'end & 3'end)
# for this purpose, current coordiantes (from input last exons) is sufficient.
# but for bleedthrough events, this will contain the complete last exon. So need to return to quant last exons to get the unique regions. 

# novel_le_rep_bleed - extract transcript_id (i.e. represents selected isoform) & downstream le_id (le_id_quant)
novel_rep_bleed_tx2le = novel_le_rep_bleed.as_df()[["transcript_id", "le_id_quant", "Name"]].rename(columns={"le_id_quant": "le_id"})
novel_rep_bleed_tx2le



,transcript_id,le_id,Name
0,PAPA.TDP-1.2120.4,ENSG00000082497.12_1,ENSG00000082497.12_1|SERTAD4|bleedthrough|back...
1,PAPA.CTRL-2.2002.3,ENSG00000082497.12_1,ENSG00000082497.12_1|SERTAD4|bleedthrough|back...
2,PAPA.ctrl_ctrl_4.2339.3,ENSG00000082497.12_1,ENSG00000082497.12_1|SERTAD4|bleedthrough|back...
3,PAPA.TDP43-G_S7.362.1,ENSG00000117682.17_3,ENSG00000117682.17_3|DHDDS|bleedthrough|backgr...
4,PAPA.doxconc_NT_0_3.354.1,ENSG00000117682.17_6,ENSG00000117682.17_6|DHDDS|bleedthrough|backgr...
...,...,...,...
307,PAPA.TDP43-G_S7.20350.2,ENSG00000180182.11_3,ENSG00000180182.11_3|MED14|bleedthrough|cryptic
308,PAPA.TDP-2.25089.1,ENSG00000182518.14_1,ENSG00000182518.14_1|FAM104B|bleedthrough|back...
309,PAPA.doxconc_NT_0_3.19568.4,ENSG00000196459.15_1,ENSG00000196459.15_1|TRAPPC2|bleedthrough|back...
310,PAPA.Cont-D_S4.20482.2,ENSG00000197021.9_2,ENSG00000197021.9_2|EOLA2|bleedthrough|background


In [14]:
for k,v in rep_choices_bleed.items():
    if "ENSG00000082497.12_1" in v:
        print(k)

atlas_max_datasets


In [15]:
# subset to representative bleedthrough txs, joining in Name information
# 5'coord = 1st cooridnate of intron (i.e. where spliced occurs)
# 3'coord = final coordinate of last exon
novel_le_rep_bleed_quant = quant_uniq_le.apply(lambda df: df.merge(novel_rep_bleed_tx2le, on=["transcript_id", "le_id"], how="inner"))

novel_le_rep_bleed_quant[["Score", "Name", "transcript_id", "le_id"]].drop_duplicate_positions()



,Chromosome,Start,End,Score,Strand,Name,transcript_id,le_id
0,chr1,210241557,210245767,.,+,ENSG00000082497.12_1|SERTAD4|bleedthrough|back...,PAPA.TDP-1.2120.4,ENSG00000082497.12_1
1,chr1,210245927,210246847,.,+,ENSG00000082497.12_1|SERTAD4|bleedthrough|back...,PAPA.TDP-1.2120.4,ENSG00000082497.12_1
2,chr1,210245927,210246813,.,+,ENSG00000082497.12_1|SERTAD4|bleedthrough|back...,PAPA.CTRL-2.2002.3,ENSG00000082497.12_1
3,chr1,210245927,210246804,.,+,ENSG00000082497.12_1|SERTAD4|bleedthrough|back...,PAPA.ctrl_ctrl_4.2339.3,ENSG00000082497.12_1
4,chr1,26438284,26439856,.,+,ENSG00000117682.17_3|DHDDS|bleedthrough|backgr...,PAPA.TDP43-G_S7.362.1,ENSG00000117682.17_3
...,...,...,...,...,...,...,...,...
356,chrX,40653641,40654363,.,-,ENSG00000180182.11_3|MED14|bleedthrough|cryptic,PAPA.TDP43-G_S7.20350.2,ENSG00000180182.11_3
357,chrX,55142867,55146180,.,-,ENSG00000182518.14_1|FAM104B|bleedthrough|back...,PAPA.TDP-2.25089.1,ENSG00000182518.14_1
358,chrX,149931286,149932588,.,-,ENSG00000197021.9_2|EOLA2|bleedthrough|background,PAPA.Cont-D_S4.20482.2,ENSG00000197021.9_2
359,chrX,131823777,131825221,.,-,ENSG00000213468.7_1|FIRRE|bleedthrough|cryptic,PAPA.TDP43_ctrl_2.26896.10,ENSG00000213468.7_1


### Repeat for annotated

In [16]:
# get le_ids in which all transcripts are annotated
annotated_le_ids = dexseq_df.groupby("le_id").filter(lambda df: (df.annot_status == "annotated").all()).le_id
le_id_bleed_annot = set(le_id_bleed).intersection(set(annotated_le_ids))
le_id_spliced_annot = set(le_id_spliced).intersection(set(annotated_le_ids))
print(f"Number of annotated bleedthrough last exons (assessed by DEXSeq) - {len(le_id_bleed_annot)}")
print(f"Number of annotated spliced last exons (assessed by DEXSeq) - {len(le_id_spliced_annot)}")

Number of annotated bleedthrough last exons (assessed by DEXSeq) - 2288
Number of annotated spliced last exons (assessed by DEXSeq) - 3561


In [17]:
# subset quantification GTF for spliecd last exons 
ref_le_spliced = quant_uniq_le.subset(lambda df: df.le_id.isin(le_id_spliced_annot))
# select representative isoform for each last exon ID (most distal 3'end)
ref_le_spliced = select_rep_prox_site(ref_le_spliced, "le_id")

# subset quantification GTF for bleedthrough last exons 
ref_le_bleed = quant_uniq_le.subset(lambda df: df.le_id.isin(le_id_bleed_annot))
# select representative isoform for each last exon ID (most distal 3'end)
ref_le_bleed = select_rep_prox_site(ref_le_bleed, "le_id")

In [18]:
# assign cryptic status for annotatesd spliced last exons
ref_le_spliced = ref_le_spliced.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids, ns_le_ids, id_col="le_id"))
ref_le_spliced.cryptic_status.value_counts()


cryptic_status
NULL          2275
background    2176
cryptic         39
Name: count, dtype: int64

In [19]:
# assign cryptic status for annotatesd bleedthrough last exons
ref_le_bleed = ref_le_bleed.assign("cryptic_status", lambda df: _define_cryptic_status(df, cryp_le_ids, ns_le_ids, id_col="le_id"))
ref_le_bleed.cryptic_status.value_counts()

cryptic_status
background    1382
NULL          1261
cryptic         24
Name: count, dtype: int64

In [20]:
ref_le_spliced[["ref_gene_name"]]

,Chromosome,Start,End,Strand,ref_gene_name
0,chr1,109214546,109215153,+,SARS1
1,chr1,69873466,69875004,+,LRRC7
2,chr1,6634019,6635586,+,THAP3
3,chr1,7779625,7781432,+,VAMP3
4,chr1,243305011,243306548,+,SDCCAG8
...,...,...,...,...,...
4485,chrY,12859278,12860839,+,USP9Y
4486,chrY,18932316,18932841,-,TTTY14
4487,chrY,18872500,18872834,-,TTTY14
4488,chrY,18872500,18872834,-,TTTY14


In [21]:
# assign Name field with minimal info
ref_le_spliced = (ref_le_spliced
  .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("gene_name_ref",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         # some ref_gene_name entries have multiple gene names
         lambda df: df["ref_gene_name"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
         .assign("site_type",
                 lambda df: pd.Series(["spliced"]*len(df.index), index=df.index))
 .assign("Name",
                          lambda df: df["le_id"].str.cat(df[["gene_name_ref", "site_type", "cryptic_status"]], sep="|"))
)

ref_le_bleed = (ref_le_bleed
 .subset(lambda df: df["cryptic_status"].ne("NULL"))
 .assign("gene_name_ref",
         # list(dict.fromkeys(x.split(","))) - drops duplicates whilst preserving order
         # some ref_gene_name entries have multiple gene names
         lambda df: df["ref_gene_name"].apply(lambda x: ",".join(list(dict.fromkeys(x.split(",")))))
         )
.assign("site_type",
                 lambda df: pd.Series(["bleedthrough"]*len(df.index), index=df.index))
 .assign("Name",
                          lambda df: df["le_id"].str.cat(df[["gene_name_ref", "site_type", "cryptic_status"]], sep="|"))
)

In [22]:
# since last_exon_spliced annotations includes most distal last exon of a gene, let's compare these ALEs to non-terminal ALEs (i.e. there is a competing splicign event)
print(ref_le_spliced.event_type.value_counts())

ref_le_spliced = ref_le_spliced.subset(lambda df: df["event_type"].ne("last_exon_spliced"))
ref_le_spliced
      

event_type
last_exon_spliced                           1190
first_exon_spliced                           525
internal_exon_spliced                        247
internal_exon_extension                      106
first_exon_spliced,internal_exon_spliced     106
first_exon_extension                          41
Name: count, dtype: int64


,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,gene_id,gene_name,...,3p_extension_length,event_type,ref_gene_id,ref_gene_name,le_number,le_id,cryptic_status,gene_name_ref,site_type,Name
0,chr1,.,exon,39753286,39756769,.,+,.,ENSG00000084072.17,PPIE,...,NULL,internal_exon_spliced,ENSG00000084072.17,PPIE,2.0,ENSG00000084072.17_2,background,PPIE,spliced,ENSG00000084072.17_2|PPIE|spliced|background
1,chr1,.,exon,39763688,39763914,.,+,.,ENSG00000084072.17,PPIE,...,NULL,internal_exon_spliced,ENSG00000084072.17,PPIE,3.0,ENSG00000084072.17_3,background,PPIE,spliced,ENSG00000084072.17_3|PPIE|spliced|background
2,chr1,.,exon,170736047,170739421,.,+,.,ENSG00000116132.12,PRRX1,...,"NULL,NULL",internal_exon_spliced,ENSG00000116132.12,PRRX1,2.0,ENSG00000116132.12_2,background,PRRX1,spliced,ENSG00000116132.12_2|PRRX1|spliced|background
3,chr1,.,exon,110232597,110249124,.,+,.,ENSG00000116396.15,KCNC4,...,16527.0,internal_exon_extension,ENSG00000116396.15,KCNC4,2.0,ENSG00000116396.15_2,background,KCNC4,spliced,ENSG00000116396.15_2|KCNC4|spliced|background
4,chr1,.,exon,32855923,32858875,.,+,.,ENSG00000116497.18,S100PBP,...,NULL,first_exon_spliced,ENSG00000116497.18,S100PBP,3.0,ENSG00000116497.18_3,background,S100PBP,spliced,ENSG00000116497.18_3|S100PBP|spliced|background
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,chrX,.,exon,72274254,72274435,.,-,.,ENSG00000198034.11,RPS4X,...,NULL,internal_exon_spliced,ENSG00000198034.11,RPS4X,1.0,ENSG00000198034.11_1,background,RPS4X,spliced,ENSG00000198034.11_1|RPS4X|spliced|background
1021,chrX,.,exon,130981589,130981856,.,-,.,ENSG00000228659.1,LINC01201,...,NULL,first_exon_spliced,ENSG00000228659.1,LINC01201,3.0,ENSG00000228659.1_3,background,LINC01201,spliced,ENSG00000228659.1_3|LINC01201|spliced|background
1022,chrX,.,exon,149527590,149528005,.,-,.,"ENSG00000241489.8,ENSG00000241769.7",EOLA1-DT,...,"NULL,NULL,NULL",internal_exon_spliced,"ENSG00000241489.8,ENSG00000241769.7",EOLA1-DT,1.0,"ENSG00000241489.8,ENSG00000241769.7_1",background,EOLA1-DT,spliced,"ENSG00000241489.8,ENSG00000241769.7_1|EOLA1-DT..."
1023,chrY,.,exon,18872500,18872834,.,-,.,ENSG00000176728.10,TTTY14,...,NULL,first_exon_spliced,ENSG00000176728.10,TTTY14,3.0,ENSG00000176728.10_3,background,TTTY14,spliced,ENSG00000176728.10_3|TTTY14|spliced|background


In [23]:
# combined output BEDs
spliced_out_bed = pr.concat([novel_le_rep_spliced[["Score", "Name"]].drop_duplicate_positions(),
                             ref_le_spliced[["Score", "Name"]].drop_duplicate_positions()])

bleed_out_bed = pr.concat([novel_le_rep_bleed_quant[["Score", "Name"]].drop_duplicate_positions(),
                             ref_le_bleed[["Score", "Name"]].drop_duplicate_positions()])

spliced_out_bed.subset(lambda df: df.Name.str.contains("cryptic$"))

# novel_le_rep_bleed_quant[["Score", "Name"]].drop_duplicate_positions()
# novel_le_rep_spliced[["Score", "Name"]].drop_duplicate_positions()


# generate output bed file of proximal and distal PAS
# utr3_out_bed = pr.concat([sel_le_d3utr_prox[["Score", "Name"]],
#                           sel_novel_le_d3utr[["Score", "Name"]]]
#                           )

# print(len(utr3_out_bed))
# # drop duplicate entries
# utr3_out_bed = utr3_out_bed.apply(lambda df: df.drop_duplicates()).sort()

# utr3_out_bed

,Chromosome,Start,End,Score,Strand,Name
0,chr1,76871267,76871821,.,+,ENSG00000117069.15_2|ST6GALNAC5|spliced|cryptic
1,chr1,245464258,245471621,.,+,ENSG00000162849.16_2|KIF26B|spliced|cryptic
2,chr1,1616614,1619210,.,+,ENSG00000197530.13_1|MIB2|spliced|cryptic
3,chr1,112540178,112542090,.,-,ENSG00000007341.19_4|ST7L|spliced|cryptic
4,chr1,93783620,93784191,.,-,ENSG00000137936.18_1|BCAR3|spliced|cryptic
5,chr1,33591806,33592489,.,-,ENSG00000121904.19_3|CSMD2|spliced|cryptic
6,chr1,20775449,20776596,.,-,ENSG00000127483.19_1|HP1BP3|spliced|cryptic
7,chr1,163304210,163305313,.,-,ENSG00000232995.7_2|ENSG00000232995|spliced|cr...
8,chr2,135060280,135063187,.,+,ENSG00000115839.19_2|RAB3GAP1|spliced|cryptic
9,chr2,205776245,205779714,.,+,ENSG00000118257.17_5|NRP2|spliced|cryptic


In [24]:
bleed_out_bed.subset(lambda df: df.Name.str.contains("cryptic$"))

,Chromosome,Start,End,Score,Strand,Name
0,chr1,45013791,45013889,.,+,ENSG00000126088.14_1|UROD|bleedthrough|cryptic
1,chr1,1616614,1619210,.,+,ENSG00000197530.13_1|MIB2|bleedthrough|cryptic
2,chr1,112540178,112540778,.,-,ENSG00000007341.19_4|ST7L|bleedthrough|cryptic
3,chr1,112540872,112541950,.,-,ENSG00000007341.19_4|ST7L|bleedthrough|cryptic
4,chr1,20775449,20776596,.,-,ENSG00000127483.19_1|HP1BP3|bleedthrough|cryptic
5,chr2,225651535,225658565,.,+,ENSG00000144460.13_2|NYAP2|bleedthrough|cryptic
6,chr2,216451064,216451601,.,+,ENSG00000138375.13_1|SMARCAL1|bleedthrough|cry...
7,chr2,197630635,197631010,.,-,ENSG00000162944.11_1|RFTN2|bleedthrough|cryptic
8,chr3,52414587,52416161,.,+,ENSG00000010318.22_2|PHF7|bleedthrough|cryptic
9,chr3,113425851,113426123,.,-,ENSG00000206530.11_1|CFAP44|bleedthrough|cryptic


In [25]:
if not os.path.exists("../processed/iclip_regions"):
    os.makedirs("../processed/iclip_regions")

#### TEMPORARY - work out why duplicate regions/multiple PAS coords for some last exons. To prevent duplciats slect longest per interval
spliced_out_bed = select_rep_prox_site(spliced_out_bed, "Name")
bleed_out_bed = select_rep_prox_site(bleed_out_bed, "Name")

spliced_out_bed.to_bed("../processed/iclip_regions/2023-06-26_papa_cryptic_spliced.last_exons.bed")
# also make one with just polYA sites
spliced_out_bed.three_end().to_bed("../processed/iclip_regions/2023-06-26_papa_cryptic_spliced.pas.bed")
# also first coordinate of exon (i.e. interval ending at this coordinate = splice site end, or substract 1 (strand-aware) to get final nucleotide of splice site)
spliced_out_bed.five_end().to_bed("../processed/iclip_regions/2023-06-26_papa_cryptic_spliced.le_start.bed")

bleed_out_bed.to_bed("../processed/iclip_regions/2023-06-26_papa_cryptic_bleedthrough.last_exons.bed")
# also make one with just polYA sites
bleed_out_bed.three_end().to_bed("../processed/iclip_regions/2023-06-26_papa_cryptic_bleedthrough.pas.bed")
# also first coordinate of exon (i.e. 1st nucleotide of intron
bleed_out_bed.five_end().to_bed("../processed/iclip_regions/2023-06-26_papa_cryptic_bleedthrough.le_start.bed")